<a href="https://colab.research.google.com/github/felixM2020/ecn_t10_scorecard_scraper/blob/main/bowling_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bowling Scraper


Install and open BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import requests
page = requests.get('https://www.sportsadda.com/cricket/series/european-championship-2021-1991/scores-fixtures')
soup = BeautifulSoup(page.content,  'html.parser')

Get all matches links

In [ ]:
matches = []
card_list = soup.find('div', {'class': 'card-list'})
#print(card_list)

for box in soup.find_all('div', {'class': 'card-list'}):
  for a in box.find_all('a', {'class':'card-item recent'}):
    link = a.get('href')
    if 'cricket/scores-fixtures' not in link:
      print(link)
    else:
      matches.append(link)


Define function to interpret these links

In [ ]:
def complete_url(link):
  return 'https://www.sportsadda.com' + link

Set up Selenium

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

###Function to scrape data from a single link

Helper Functions

In [ ]:
from selenium.webdriver.common.by import By
import time
import pandas as pd

def get_extra_details(driver, i):
  out = {}
  teams = get_teams(driver)
  out['Team'] = bowling_team(teams, i)
  out['Opponent'] = batting_team(teams,i)
  out['Date'] = get_date(driver)
  out['Result'] = result_string(driver)
  return out

def get_date(driver):
  date = driver.find_element(By.XPATH, '//span[@class = "match-time"]')
  return date.text


def opposite_index(i):
  if i == 0:
    return 1
  return 0

def get_teams(driver):
  headers = driver.find_elements(By.XPATH, '//ul[@class = "tabs scorecard-sub-tab"]/li/span[@class ="fullname"]')
  return [h.text for h in headers]

def batting_team(teams, i):
  return teams[opposite_index(i)]
def bowling_team(teams, i):
  return teams[i]

def result_string(driver):
  result = driver.find_element(By.XPATH, '//span[@class = "team-status"]')
  return result.text


def scrape_table(table, extra_details):
  header = table.find_element(By.XPATH, './/div[@class = "table-head"]//div[@class = "table-row"]')
  output = pd.DataFrame(get_rows(table, extra_details) ,columns = get_cols(header, extra_details))
  return output
  
def get_rows(table, extra_details):
  out = []
  for row in table.find_elements(By.XPATH, './/div[@class = "table-body"]//div[@class = "table-row"]'):
    add = list(extra_details.values())
    for div in row.find_elements(By.XPATH, './div'):
      add.append(div.text)
    out.append(add)
  return out

def get_cols(header, extra_details):
  cols = list(extra_details.keys())
  for div in header.find_elements(By.XPATH, './div'):
    cols.append(div.text)
  return cols

Function

In [ ]:
def scrape(link):
  driver.get(complete_url(link))
  time.sleep(2)
  out = []
  for i in range(2):
    li = driver.find_elements(By.XPATH, '//ul[@class = "tabs scorecard-sub-tab"]/li')
    try:
      li[i].click()
    except:
      pass
    extra_details = get_extra_details(driver, i)
    table = driver.find_element(By.XPATH, '//div[@class = "table bowling-table"]')
    out.append(scrape_table(table, extra_details))
  try:  
    return pd.concat(out)
  except:
    try: return out[0]
    except: pass

  

Iterate across all games

In [ ]:
frames = [scrape(matches[0])]
for i in  range(1, len(matches)):
  try:
    df = scrape(matches[i])
    frames.append(df)
    print(i)
  except:
    print(matches[i])

df = pd.concat(frames)


In [20]:
df

,Team,Opponent,Date,Result,Bowling,O,M,R,W,NB,WD,E/R
0,Belgium,England XI,| Friday 8th October | 15:00,England XI beat Belgium by 10 wickets,Murid Ekrami,1.5,0,35,0,0,1,19.09
1,Belgium,England XI,| Friday 8th October | 15:00,England XI beat Belgium by 10 wickets,Fahim Bhatti,2,0,37,0,0,1,18.50
2,Belgium,England XI,| Friday 8th October | 15:00,England XI beat Belgium by 10 wickets,Waqas Raja,1,0,28,0,1,0,28
3,Belgium,England XI,| Friday 8th October | 15:00,England XI beat Belgium by 10 wickets,Aziz Mohammad,2,0,18,0,0,3,9
4,Belgium,England XI,| Friday 8th October | 15:00,England XI beat Belgium by 10 wickets,Khalid Ahmadzai,2,0,30,0,1,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1,Belgium,Spain,| Monday 13th September | 07:00,Belgium beat Spain by 4 wickets,Adeel Raja,1,0,17,0,1,0,17
2,Belgium,Spain,| Monday 13th September | 07:00,Belgium beat Spain by 4 wickets,Kuldeep Lal,2,0,6,3,0,0,3
3,Belgium,Spain,| Monday 13th September | 07:00,Belgium beat Spain by 4 wickets,Yasir Ali,2,0,28,0,0,0,14
4,Belgium,Spain,| Monday 13th September | 07:00,Belgium beat Spain by 4 wickets,Paul Hennessy,1.4,0,22,2,0,1,13.20


Save df to csv file

In [ ]:
csv_title = 't10_ecn_bowling.csv'
from google.colab import files
df.to_csv(csv_title) 
files.download(csv_title)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>